# Scrape NBA Teams Regular season game logs.
- Find b2b games in each year, 3 in 4, 4 in 5
- Using time series analysis

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime

import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/amos/Desktop/chromedriver"

In [3]:
browser = webdriver.Chrome(executable_path= chromedriver)

In [4]:
stat = "http://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season=%s&SeasonType=Regular+Season&Sorter=PTS"


In [5]:
browser.get('http://stats.nba.com/league/team/#!/gamelogs/')
HTML = browser.page_source
years = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()
years_96_2016 = years[1:21]
years_96_2016

[u'2015-16',
 u'2014-15',
 u'2013-14',
 u'2012-13',
 u'2011-12',
 u'2010-11',
 u'2009-10',
 u'2008-09',
 u'2007-08',
 u'2006-07',
 u'2005-06',
 u'2004-05',
 u'2003-04',
 u'2002-03',
 u'2001-02',
 u'2000-01',
 u'1999-00',
 u'1998-99',
 u'1997-98',
 u'1996-97']

In [8]:

for y in years_96_2016:
    print y
    link = stat % y
    browser.get(link)
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, 'lxml')
    get_text = soup.findAll('pre')[0].getText().encode('utf-8')
    json_text = json.loads(get_text)
    
    df = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers']) 
    
    df_date = df[['TEAM_ABBREVIATION', 'GAME_DATE']]
    df_date.GAME_DATE = pd.to_datetime(df_date.GAME_DATE)
    df_date.sort_values('GAME_DATE', inplace=True)
    df_date.index = df_date.GAME_DATE
    
    # Goal: create a Dictionary {team: [count of B2B, count of 3 gms in 4 days, 4gms in 5 days]}
    teams = df.TEAM_ABBREVIATION.unique()
    # 1. set up dictionary 
    team_dict = {}
    # 2. iterate through each nba team 
    for team in teams:
        # date of each game played per team
        game_day = df_date[df_date.TEAM_ABBREVIATION == team].index
        b2b = 0
        three_gms_in4 = 0
        four_gms_in5 = 0

        # 2. Count of B2B games
        for i in range(1, game_day.shape[0]):
            diff_2 = (game_day[i]-game_day[i-1]).days
            if  diff_2 == 1:
                b2b += 1

        # subtract days of every three consecutive game days and add count if the difference is 4
        # assuming no team plays 3 games in consecutive days
        for i in range(2, game_day.shape[0]):
            diff_4 = (game_day[i]-game_day[i-2]).days +1
            if  diff_4 == 4:
                three_gms_in4 += 1

        for i in range(3, game_day.shape[0]):    
            diff_5 = (game_day[i]-game_day[i-3]).days + 1
            if  diff_5 == 5:
                four_gms_in5 += 1
        team_dict[team] = [b2b , three_gms_in4, four_gms_in5]
        
    df_final = pd.DataFrame.from_dict(team_dict, orient='index')
    df_final.rename(columns={0:'B2B_COUNT', 1:'3GMS_IN_4DAYS', 2:'4GMS_IN_5DAYS'}, inplace=True)
    df_final['SEASON'] = y[:4]
    df_final['TEAM_ABBREVIATION'] = df_final.index
    
    save_csv = '/Users/amos/Desktop/DSI-SF-2-amoskim25/copy_capstone/datasets/team_schedule/team_schedule%s.csv' % y
    df_final.to_csv(save_csv)


2015-16


/Users/amos/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97


In [122]:
# # Need to create a dataframe of ['SEASON', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'COUNT_B2B']

# df_date = df[['TEAM_ABBREVIATION', 'GAME_DATE']]
# df_date.GAME_DATE = pd.to_datetime(df_date.GAME_DATE)
# df_date.sort_values('GAME_DATE', inplace=True)
# df_date.index = df_date.GAME_DATE

In [121]:
# # Goal: create a Dictionary {team: [count of B2B, count of 3 gms in 4 days, 4gms in 5 days]}

# # 1. set up dictionary 
# team_dict = {}
# # 2. iterate through each nba team 
# for team in teams:
#     # date of each game played per team
#     game_day = df_date[df_date.TEAM_ABBREVIATION == team].index
#     b2b = 0
#     three_gms_in4 = 0
#     four_gms_in5 = 0
    
#     # 2. Count of B2B games
#     for i in range(1, game_day.shape[0]):
#         diff_2 = (game_day[i]-game_day[i-1]).days
#         if  diff_2 == 1:
#             b2b += 1

#     # subtract days of every three consecutive game days and add count if the difference is 4
#     # assuming no team plays 3 games in consecutive days
#     for i in range(2, game_day.shape[0]):
#         diff_4 = (game_day[i]-game_day[i-2]).days +1
#         if  diff_4 == 4:
#             three_gms_in4 += 1

#     for i in range(3, game_day.shape[0]):    
#         diff_5 = (game_day[i]-game_day[i-3]).days + 1
#         if  diff_5 == 5:
#             four_gms_in5 += 1
#     team_dict[team] = [b2b , three_gms_in4, four_gms_in5]
